In [33]:
"""Notebook de soporte para el spike de Gràcia.

Objetivo:
- Descargar/asegurar los datos de Portal Dades (incluyendo el indicador de alquiler b37xv8wcjh).
- Reutilizar `prepare_portaldades_precios` para generar un DataFrame de alquiler.
- Filtrar Gràcia 2020-2025.
- Exportar `spike-data-validation/data/raw/ine_alquiler_gracia.csv`.
"""
import os
  # tu valor
# Mostrar solo si la variable está definida, pero sin imprimir el valor por seguridad
if "PORTALDADES_CLIENT_ID" in os.environ:
    print("La variable de entorno PORTALDADES_CLIENT_ID está definida.")
else:
    print("La variable de entorno PORTALDADES_CLIENT_ID NO está definida.")
import sys
from pathlib import Path

# Ruta absoluta a la raíz del proyecto
PROJECT_ROOT = Path("/Users/adrianiraeguialvear/Projects/barcelona-housing-demographics-analyzer")
sys.path.insert(0, str(PROJECT_ROOT))

import pandas as pd

from src.database_setup import create_connection, ensure_database_path
from src.etl.transformations.enrichment import prepare_portaldades_precios
from src.extraction import PortalDadesExtractor
from src.extraction.base import DATA_RAW_DIR


La variable de entorno PORTALDADES_CLIENT_ID está definida.


In [34]:
# 1. Asegurar descarga de indicadores de Habitatge (incluye b37xv8wcjh)

portal_extractor = PortalDadesExtractor(output_dir=DATA_RAW_DIR)

indicadores, archivos = portal_extractor.extraer_y_descargar_habitatge(
    descargar=True,
    formato="CSV",
    max_pages=None,
)

len(indicadores), list(archivos.keys())[:5]


2025-12-16 16:40:26 - src.extraction.base - INFO - Extrayendo indicadores del Portal de Dades usando API REST
2025-12-16 16:40:26 - src.extraction.base - INFO - Total de indicadores disponibles: 141
2025-12-16 16:40:26 - src.extraction.base - INFO - Procesando 15 páginas (máximo todas)
2025-12-16 16:40:28 - src.extraction.base - INFO - Página 1 (start=0): 10 indicadores encontrados (total acumulado: 10)
2025-12-16 16:40:30 - src.extraction.base - INFO - Página 2 (start=10): 10 indicadores encontrados (total acumulado: 20)
2025-12-16 16:40:32 - src.extraction.base - INFO - Página 3 (start=20): 10 indicadores encontrados (total acumulado: 30)


KeyboardInterrupt: 

In [35]:
import sqlite3
from pathlib import Path

for path in [
    Path("data/processed/database.db"),
    Path("data/processed/database_backup_baseline.db"),
]:
    print("====", path)
    conn = sqlite3.connect(path)
    try:
        tables = conn.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()
        print(tables)
    finally:
        conn.close()

==== data/processed/database.db
[]
==== data/processed/database_backup_baseline.db
[]


In [36]:
from pathlib import Path
import pandas as pd

# Ruta absoluta a la raíz del proyecto
PROJECT_ROOT = Path("/Users/adrianiraeguialvear/Projects/barcelona-housing-demographics-analyzer")

loc_path = PROJECT_ROOT / "data/processed/barrio_location_ids.csv"
master_path = PROJECT_ROOT / "data/processed/barcelona_housing_master_table.csv"

print("Existe barrio_location_ids:", loc_path.exists())
print("Existe master_table:", master_path.exists())

# Usamos barrio_location_ids como fuente principal de dim_barrios
dim_barrios = pd.read_csv(loc_path)

dim_barrios.head(), dim_barrios.dtypes

Existe barrio_location_ids: True
Existe master_table: True


(   barrio_id                          barrio_nombre  \
 0          1                               el Raval   
 1          2                         el Barri Gòtic   
 2          3                         la Barceloneta   
 3          4  Sant Pere, Santa Caterina i la Ribera   
 4          5                          el Fort Pienc   
 
         barrio_nombre_normalizado  distrito_id distrito_nombre  municipio  \
 0                         elraval            1    Ciutat Vella  Barcelona   
 1                    elbarrigotic            1    Ciutat Vella  Barcelona   
 2                   labarceloneta            1    Ciutat Vella  Barcelona   
 3  santperesantacaterinailaribera            1    Ciutat Vella  Barcelona   
 4                     elfortpienc            2        Eixample  Barcelona   
 
   ambito  codi_districte  codi_barri  \
 0  barri               1           1   
 1  barri               1           2   
 2  barri               1           3   
 3  barri               1   

In [37]:
# 3. Preparar precios Portal Dades (venta y alquiler)
from datetime import datetime, timezone
reference_time = datetime.now(timezone.utc)

reference_time = datetime.utcnow()
portaldades_dir = DATA_RAW_DIR / "portaldades"
metadata_file = portaldades_dir / "indicadores_habitatge.csv"

venta_df, alquiler_df = prepare_portaldades_precios(
    portaldades_dir=portaldades_dir,
    dim_barrios=dim_barrios,
    reference_time=reference_time,
    metadata_file=metadata_file if metadata_file.exists() else None,
)

len(venta_df), len(alquiler_df)


/var/folders/8t/1rzjlj6n3rj3v08skl2pwjbm0000gn/T/ipykernel_70291/441113592.py:5: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  reference_time = datetime.utcnow()


(35740, 4002)

In [38]:
# 4. Filtrar alquiler para el distrito de Gràcia (2020-2025)

# Identificar barrio_id de Gràcia usando la columna 'distrito_nombre'
mask_gracia = dim_barrios["distrito_nombre"].str.strip().str.lower() == "gràcia".lower()

gracia_ids = dim_barrios.loc[mask_gracia, "barrio_id"].unique()

alquiler_gracia = alquiler_df[
    (alquiler_df["barrio_id"].isin(gracia_ids))
    & (alquiler_df["anio"].between(2020, 2025))
].copy()

len(gracia_ids), gracia_ids, alquiler_gracia["anio"].min(), alquiler_gracia["anio"].max(), len(alquiler_gracia)


(5, array([28, 29, 30, 31, 32]), np.int64(2020), np.int64(2025), 140)

In [39]:
# 5. Inspección rápida y export a CSV del spike

alquiler_gracia.head()


,barrio_id,anio,periodo,trimestre,precio_m2_venta,precio_mes_alquiler,dataset_id,source,etl_loaded_at
2159,32,2020,2020,<NA>,<NA>,967.938095,b37xv8wcjh,portaldades,2025-12-16T15:40:45.186493
2161,32,2020,2020,<NA>,<NA>,953.072055,b37xv8wcjh,portaldades,2025-12-16T15:40:45.186493
2162,28,2020,2020,<NA>,<NA>,966.437000,b37xv8wcjh,portaldades,2025-12-16T15:40:45.186493
2164,28,2020,2020,<NA>,<NA>,982.490248,b37xv8wcjh,portaldades,2025-12-16T15:40:45.186493
2167,31,2020,2020,<NA>,<NA>,963.608985,b37xv8wcjh,portaldades,2025-12-16T15:40:45.186493


In [40]:
# 6. Exportar dataset de alquiler Gràcia 2020-2025 para el spike

output_path = Path("spike-data-validation/data/raw/ine_alquiler_gracia.csv")
output_path.parent.mkdir(parents=True, exist_ok=True)

alquiler_gracia.to_csv(output_path, index=False, encoding="utf-8")
output_path, len(alquiler_gracia)


(PosixPath('spike-data-validation/data/raw/ine_alquiler_gracia.csv'), 140)

In [41]:
# 7. Listar ficheros CSV descargados de Portal Dades

from pathlib import Path

portaldades_dir = DATA_RAW_DIR / "portaldades"
portaldades_files = sorted(portaldades_dir.glob("portaldades_*.csv"))

len(portaldades_files), portaldades_files[:5]


(141,
 [PosixPath('/Users/adrianiraeguialvear/Projects/barcelona-housing-demographics-analyzer/data/raw/portaldades/portaldades_Capital_prestat_a_les_hipoteques_constituïdes_per_naturalesa_de_la_finca_j16exwlpmd.csv'),
  PosixPath('/Users/adrianiraeguialvear/Projects/barcelona-housing-demographics-analyzer/data/raw/portaldades/portaldades_Capital_prestat_a_les_hipoteques_de_finques_urbanes_constituïdes_clqf3778yr.csv'),
  PosixPath('/Users/adrianiraeguialvear/Projects/barcelona-housing-demographics-analyzer/data/raw/portaldades/portaldades_Capital_prestat_a_les_hipoteques_de_finques_urbanes_constituïdes_per_entitat_prestadora_mxlmngg0bt.csv'),
  PosixPath('/Users/adrianiraeguialvear/Projects/barcelona-housing-demographics-analyzer/data/raw/portaldades/portaldades_Capital_prestat_a_les_hipoteques_dhabitatge_constituïdes_evgoeymapo.csv'),
  PosixPath('/Users/adrianiraeguialvear/Projects/barcelona-housing-demographics-analyzer/data/raw/portaldades/portaldades_Edat_mitjana_de_les_edificaci

In [42]:
# 8. Inspección de estructura de un dataset concreto (ejemplo)

example_file = portaldades_files[0]
print("Ejemplo de fichero:", example_file)

example_df = pd.read_csv(example_file)
example_df.head(), example_df.dtypes, example_df.columns


Ejemplo de fichero: /Users/adrianiraeguialvear/Projects/barcelona-housing-demographics-analyzer/data/raw/portaldades/portaldades_Capital_prestat_a_les_hipoteques_constituïdes_per_naturalesa_de_la_finca_j16exwlpmd.csv


(           Dim-00:TEMPS Dim-01:TERRITORI  Dim-01:TERRITORI (order)  \
 0  2008-01-01T00:00:00Z        Barcelona                        -1   
 1  2008-01-01T00:00:00Z        Barcelona                        -1   
 2  2008-02-01T00:00:00Z        Barcelona                        -1   
 3  2008-02-01T00:00:00Z        Barcelona                        -1   
 4  2008-03-01T00:00:00Z        Barcelona                        -1   
 
   Dim-01:TERRITORI (type) Dim-02:NATURALESA DE LA FINCA     VALUE  
 0                Municipi                    Habitatges  432971.0  
 1                Municipi                Altres finques  246483.0  
 2                Municipi                Altres finques  259436.0  
 3                Municipi                    Habitatges  389580.0  
 4                Municipi                Altres finques  391912.0  ,
 Dim-00:TEMPS                      object
 Dim-01:TERRITORI                  object
 Dim-01:TERRITORI (order)           int64
 Dim-01:TERRITORI (type)       

In [43]:
# 9. Ejecutar prepare_portaldades_precios sobre todos los CSV descargados

from datetime import datetime
from src.etl.transformations.enrichment import prepare_portaldades_precios

reference_time = datetime.utcnow()
metadata_file = portaldades_dir / "indicadores_habitatge.csv"

venta_df, alquiler_df = prepare_portaldades_precios(
    portaldades_dir=portaldades_dir,
    dim_barrios=dim_barrios,
    reference_time=reference_time,
    metadata_file=metadata_file if metadata_file.exists() else None,
)

len(venta_df), len(alquiler_df)


/var/folders/8t/1rzjlj6n3rj3v08skl2pwjbm0000gn/T/ipykernel_70291/1808474615.py:6: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  reference_time = datetime.utcnow()


(35740, 4002)

In [44]:
# 11. Transformar y estructurar datos de VENTA para Gràcia 2020-2025

from pathlib import Path

# Identificar barrio_id del distrito de Gràcia
mask_gracia = dim_barrios["distrito_nombre"].str.strip().str.lower() == "gràcia".lower()

gracia_ids = dim_barrios.loc[mask_gracia, "barrio_id"].unique()
print("Barrios de Gràcia (barrio_id):", gracia_ids)

# Filtrar venta_df a Gràcia y periodo 2020-2025
gracia_venta = venta_df[
    (venta_df["barrio_id"].isin(gracia_ids))
    & (venta_df["anio"].between(2020, 2025))
].copy()

print("Registros Gràcia 2020-2025 (venta):", len(gracia_venta))
print("Años únicos:", sorted(gracia_venta["anio"].unique().tolist()))
print("Nº barrios únicos:", gracia_venta["barrio_id"].nunique())

# Renombrar y seleccionar columnas mínimas para el spike
gracia_venta = gracia_venta.rename(columns={"precio_m2_venta": "precio_m2"})
cols = [
    "barrio_id",
    "anio",
    "periodo",
    "trimestre",
    "precio_m2",
    "dataset_id",
    "source",
]

gracia_venta = gracia_venta[cols].copy()

gracia_venta.head()


Barrios de Gràcia (barrio_id): [28 29 30 31 32]
Registros Gràcia 2020-2025 (venta): 1268
Años únicos: [2020, 2021, 2022, 2023, 2024, 2025]
Nº barrios únicos: 5


,barrio_id,anio,periodo,trimestre,precio_m2,dataset_id,source
2065,28,2020,2020,<NA>,3773.74,u25rr7oxh6,portaldades
2077,31,2020,2020,<NA>,4807.18,u25rr7oxh6,portaldades
2093,30,2020,2020,<NA>,4481.47,u25rr7oxh6,portaldades
2125,29,2020,2020,<NA>,3328.85,u25rr7oxh6,portaldades
2127,30,2020,2020,<NA>,4519.74,u25rr7oxh6,portaldades


In [45]:
# 12. Exportar CSV de precios Gràcia 2020-2025 para el spike

output_path = Path("spike-data-validation/data/raw/ine_precios_gracia_notebook.csv")
output_path.parent.mkdir(parents=True, exist_ok=True)

gracia_venta.to_csv(output_path, index=False, encoding="utf-8")

output_path, len(gracia_venta), gracia_venta["barrio_id"].nunique(), sorted(gracia_venta["anio"].unique().tolist())


(PosixPath('spike-data-validation/data/raw/ine_precios_gracia_notebook.csv'),
 1268,
 5,
 [2020, 2021, 2022, 2023, 2024, 2025])

In [46]:
# 10. Checks de cobertura y calidad para el spike

# Cobertura temporal y espacial
print("Cobertura venta (anios x barrios distintos):")
print(venta_df.groupby("anio")["barrio_id"].nunique())

print("\nCobertura alquiler (anios x barrios distintos):")
print(alquiler_df.groupby("anio")["barrio_id"].nunique())

# Rango de valores
print("\nDistribucion precio_m2_venta:")
print(venta_df["precio_m2_venta"].describe())

print("\nDistribucion precio_mes_alquiler:")
print(alquiler_df["precio_mes_alquiler"].describe())


Cobertura venta (anios x barrios distintos):
anio
2012    68
2013    70
2014    73
2015    73
2016    73
2017    72
2018    73
2019    73
2020    73
2021    72
2022    73
2023    73
2024    73
2025    72
Name: barrio_id, dtype: int64

Cobertura alquiler (anios x barrios distintos):
anio
2014    73
2015    73
2016    73
2017    72
2018    72
2019    73
2020    72
2021    72
2022    71
2023    71
2024    72
2025    72
Name: barrio_id, dtype: int64

Distribucion precio_m2_venta:
count    35740.000000
mean      3269.906282
std       1261.959397
min        217.170000
25%       2337.050000
50%       3111.419286
75%       4036.302326
max      18551.140000
Name: precio_m2_venta, dtype: float64

Distribucion precio_mes_alquiler:
count    4002.000000
mean      880.001322
std       287.755194
min       142.335714
25%       687.454647
50%       826.932156
75%      1004.715454
max      2261.667000
Name: precio_mes_alquiler, dtype: float64


In [47]:
import json
from pathlib import Path

LOG_DIR = Path("spike-data-validation/data/logs")
LOG_DIR.mkdir(parents=True, exist_ok=True)

df = gracia_venta  # DataFrame ya filtrado Gràcia 2020-2025

if df.empty:
    summary_199 = {
        "total_registros": 0,
        "barrios_ids": [],
        "años_unicos": [],
        "precio_m2_min": None,
        "precio_m2_max": None,
        "precio_m2_media": None,
        "cobertura_temporal": None,
        "warning": "DataFrame vacío tras filtrado para Gràcia",
    }
else:
    summary_199 = {
        "total_registros": int(len(df)),
        "barrios_ids": sorted(int(v) for v in df["barrio_id"].unique()),
        "años_unicos": sorted(int(v) for v in df["anio"].unique()),
        "precio_m2_min": float(df["precio_m2"].min()),
        "precio_m2_max": float(df["precio_m2"].max()),
        "precio_m2_media": float(df["precio_m2"].mean()),
        "cobertura_temporal": f"{int(df['anio'].min())}-{int(df['anio'].max())}",
    }

summary_path = LOG_DIR / "extraction_summary_199.json"
with open(summary_path, "w", encoding="utf-8") as f:
    json.dump(summary_199, f, indent=2, ensure_ascii=False)

summary_path, summary_199

(PosixPath('spike-data-validation/data/logs/extraction_summary_199.json'),
 {'total_registros': 1268,
  'barrios_ids': [28, 29, 30, 31, 32],
  'años_unicos': [2020, 2021, 2022, 2023, 2024, 2025],
  'precio_m2_min': 1036.5,
  'precio_m2_max': 16952.88,
  'precio_m2_media': 4035.103573401658,
  'cobertura_temporal': '2020-2025'})

In [49]:
"""
VALIDACIÓN EXHAUSTIVA: Spike Gràcia 2020-2025
Compara resultados obtenidos vs criterios de aceptación (DoD)
"""

import pandas as pd
import json
from pathlib import Path
from datetime import datetime

# ==============================================================================
# CONFIGURACIÓN
# ==============================================================================

GRACIA_CSV = Path("spike-data-validation/data/raw/ine_precios_gracia_notebook.csv")

print("Existe CSV notebook:", GRACIA_CSV.exists())

df_spike = pd.read_csv(GRACIA_CSV, encoding="utf-8")

with open("spike-data-validation/data/logs/extraction_summary_199.json", encoding="utf-8") as f:
    summary_199 = json.load(f)

len(df_spike), summary_199

SPIKE_DIR = Path("spike-data-validation")
GRACIA_CSV_NOTEBOOK = SPIKE_DIR / "data/raw/ine_precios_gracia_notebook.csv"  # CSV del notebook
SUMMARY_JSON = SPIKE_DIR / "data/logs/extraction_summary_199.json"
LOG_FILE = SPIKE_DIR / "data/logs/ine_extraction.log"

# Cargar dim_barrios (ruta correcta desde processed)
DIM_BARRIOS_CSV = Path("data/processed/barrio_location_ids.csv")

# Usar DATA_RAW_DIR si está definido, sino usar ruta relativa
try:
    portaldades_dir_check = DATA_RAW_DIR / "portaldades"
except NameError:
    portaldades_dir_check = Path("data/raw/portaldades")

# ==============================================================================
# CLASE PARA RESULTADOS
# ==============================================================================

class ValidationResult:
    def __init__(self, criterio, target, resultado, cumple, detalles=""):
        self.criterio = criterio
        self.target = target
        self.resultado = resultado
        self.cumple = cumple
        self.detalles = detalles
    
    def __repr__(self):
        icono = "✅" if self.cumple else "❌"
        return f"{icono} {self.criterio}: {self.resultado} (target: {self.target})"

# ==============================================================================
# 1. VALIDAR INFRAESTRUCTURA REUTILIZADA
# ==============================================================================

print("=" * 80)
print("🔍 VALIDACIÓN 1: INFRAESTRUCTURA REUTILIZADA")
print("=" * 80)

resultados_infra = []

# 1.1 PortalDadesExtractor
if portaldades_dir_check.exists():
    num_csvs = len(list(portaldades_dir_check.glob("portaldades_*.csv")))
    resultados_infra.append(ValidationResult(
        "PortalDadesExtractor usado",
        "CSVs descargados",
        f"{num_csvs} archivos en {portaldades_dir_check}",
        num_csvs > 0,
        "Extractor funcionó correctamente"
    ))
else:
    resultados_infra.append(ValidationResult(
        "PortalDadesExtractor usado",
        "CSVs descargados",
        "Directorio no existe",
        False,
        f"No se encontraron datos descargados en {portaldades_dir_check}"
    ))

# 1.2 prepare_portaldades_precios
# Validamos indirectamente con el CSV generado desde el notebook
if GRACIA_CSV_NOTEBOOK.exists():
    resultados_infra.append(ValidationResult(
        "prepare_portaldades_precios ejecutado",
        "Datos procesados",
        f"CSV generado: {GRACIA_CSV_NOTEBOOK.name}",
        True,
        "Función de enriquecimiento funcionó"
    ))
else:
    resultados_infra.append(ValidationResult(
        "prepare_portaldades_precios ejecutado",
        "Datos procesados",
        "CSV no encontrado",
        False,
        f"No se generó {GRACIA_CSV_NOTEBOOK}"
    ))

# 1.3 dim_barrios
# Verificar si dim_barrios ya está cargado en el notebook (flujo notebook)
dim_barrios_loaded = False
num_barrios_gracia = 0

if 'dim_barrios' in globals() and dim_barrios is not None and not dim_barrios.empty:
    # Usar el dim_barrios ya cargado en el notebook
    mask_gracia = dim_barrios["distrito_nombre"].str.strip().str.lower() == "gràcia".lower()
    num_barrios_gracia = dim_barrios[mask_gracia]["barrio_id"].nunique()
    dim_barrios_loaded = True
    resultados_infra.append(ValidationResult(
        "dim_barrios cargado",
        "5 barrios Gràcia",
        f"{num_barrios_gracia} barrios encontrados (desde notebook)",
        num_barrios_gracia >= 4,
        f"Mapeo funcional {'✓' if num_barrios_gracia == 5 else '⚠️ (revisar)'}"
    ))
elif DIM_BARRIOS_CSV.exists():
    # Fallback: cargar desde CSV si no está en memoria
    dim_barrios_check = pd.read_csv(DIM_BARRIOS_CSV)
    mask_gracia = dim_barrios_check["distrito_nombre"].str.strip().str.lower() == "gràcia".lower()
    num_barrios_gracia = dim_barrios_check[mask_gracia]["barrio_id"].nunique()
    resultados_infra.append(ValidationResult(
        "dim_barrios cargado",
        "5 barrios Gràcia",
        f"{num_barrios_gracia} barrios encontrados (desde CSV)",
        num_barrios_gracia >= 4,
        f"Mapeo funcional {'✓' if num_barrios_gracia == 5 else '⚠️ (revisar)'}"
    ))
else:
    resultados_infra.append(ValidationResult(
        "dim_barrios cargado",
        "5 barrios Gràcia",
        "No encontrado (ni en memoria ni CSV)",
        False,
        f"No se encontró dim_barrios en memoria ni {DIM_BARRIOS_CSV}"
    ))

for r in resultados_infra:
    print(r)

# ==============================================================================
# 2. VALIDAR EXTRACCIÓN DE DATOS (ISSUE #199)
# ==============================================================================

print("\n" + "=" * 80)
print("🔍 VALIDACIÓN 2: EXTRACCIÓN PORTAL DADES (ISSUE #199)")
print("=" * 80)

resultados_199 = []

GRACIA_CSV = Path("spike-data-validation/data/raw/ine_precios_gracia_notebook.csv")

if not GRACIA_CSV.exists():
    print(f"❌ ERROR: No se encontró {GRACIA_CSV}")
    print("   Por favor ejecuta primero la celda 12 que genera el CSV")
    df_spike = None
else:
    df_spike = pd.read_csv(GRACIA_CSV, encoding="utf-8")
    print(f"✅ CSV cargado: {len(df_spike)} registros")

# Cargar JSON resumen (ya existe desde la celda anterior)
summary_path = Path("spike-data-validation/data/logs/extraction_summary_199.json")
if summary_path.exists():
    with open(summary_path, encoding="utf-8") as f:
        summary_199 = json.load(f)
    print(f"✅ JSON resumen cargado: {summary_199['total_registros']} registros")
else:
    summary_199 = None
    print("❌ JSON resumen no encontrado")

# Definir variables siempre (usando df_spike o summary_199)
if df_spike is not None and not df_spike.empty:
    años_unicos = sorted(df_spike["anio"].unique())
    años_esperados = list(range(2020, 2026))
    años_ok = all(a in años_esperados for a in años_unicos)
    num_barrios = df_spike["barrio_id"].nunique()
    barrios_ids = sorted(df_spike["barrio_id"].unique())
elif summary_199 is not None:
    años_unicos = summary_199.get("años_unicos", [])
    años_esperados = list(range(2020, 2026))
    años_ok = all(a in años_esperados for a in años_unicos)
    num_barrios = len(summary_199.get("barrios_ids", []))
    barrios_ids = summary_199.get("barrios_ids", [])
else:
    años_unicos = []
    años_ok = False
    num_barrios = 0
    barrios_ids = []

# 2.2 Período 2020-2025
if df_spike is not None and not df_spike.empty:
    resultados_199.append(ValidationResult(
        "Período temporal",
        "2020-2025",
        años_unicos,
        años_ok,
        f"{'✓ Correcto' if años_ok else '⚠️ Fuera de rango'}"
    ))
    
    # 2.3 Cobertura barrios Gràcia
    resultados_199.append(ValidationResult(
        "Cobertura barrios",
        "5 barrios",
        f"{num_barrios} barrios ({barrios_ids})",
        num_barrios >= 4,  # Flexible
        f"{'✓ Completo' if num_barrios == 5 else '⚠️ Revisar dim_barrios'}"
    ))
    
    # 2.4 Columnas requeridas
    cols_requeridas = {"barrio_id", "anio", "precio_m2"}
    cols_presentes = set(df_spike.columns)
    cols_ok = cols_requeridas.issubset(cols_presentes)
    
    resultados_199.append(ValidationResult(
        "Columnas requeridas",
        str(cols_requeridas),
        f"{len(cols_presentes)} columnas: {list(cols_presentes)[:5]}...",
        cols_ok,
        f"{'✓ Todas presentes' if cols_ok else '❌ Faltan: ' + str(cols_requeridas - cols_presentes)}"
    ))
    
    # 2.5 Calidad de datos (nulos)
    nulos_criticos = df_spike[["barrio_id", "anio", "precio_m2"]].isna().mean()
    max_nulos = nulos_criticos.max()
    
    resultados_199.append(ValidationResult(
        "Calidad datos (nulos)",
        "<10% nulos",
        f"{max_nulos*100:.1f}% máximo",
        max_nulos < 0.10,
        f"{'✓ Datos limpios' if max_nulos < 0.10 else '⚠️ Revisar nulos'}"
    ))
    
    # 2.6 Rango precio_m2 coherente
    precio_min = df_spike["precio_m2"].min()
    precio_max = df_spike["precio_m2"].max()
    precio_mean = df_spike["precio_m2"].mean()
    
    # Barcelona típicamente: 2000-6000 €/m²
    precio_ok = 1000 <= precio_min and precio_max <= 10000
    
    resultados_199.append(ValidationResult(
        "Rango precio_m2",
        "1,000-10,000 €/m²",
        f"{precio_min:.0f}-{precio_max:.0f} €/m² (media: {precio_mean:.0f})",
        precio_ok,
        f"{'✓ Coherente con Barcelona' if precio_ok else '⚠️ Outliers?'}"
    ))
    
    # 2.7 Formato UTF-8
    try:
        pd.read_csv(GRACIA_CSV_NOTEBOOK, encoding="utf-8")
        resultados_199.append(ValidationResult(
            "Formato CSV",
            "UTF-8",
            "Lectura exitosa",
            True,
            "✓ Encoding correcto"
        ))
    except Exception as e:
        resultados_199.append(ValidationResult(
            "Formato CSV",
            "UTF-8",
            f"Error: {e}",
            False,
            "❌ Problema de encoding"
        ))

for r in resultados_199:
    print(r)

# ==============================================================================
# 3. VALIDAR TRAZABILIDAD (LOGS + JSON)
# ==============================================================================

print("\n" + "=" * 80)
print("🔍 VALIDACIÓN 3: TRAZABILIDAD Y DOCUMENTACIÓN")
print("=" * 80)

resultados_traza = []

# 3.1 JSON resumen
if SUMMARY_JSON.exists():
    with open(SUMMARY_JSON, encoding="utf-8") as f:
        summary_199 = json.load(f)
    
    resultados_traza.append(ValidationResult(
        "JSON resumen generado",
        "extraction_summary_199.json",
        f"Archivo creado: {SUMMARY_JSON.stat().st_size} bytes",
        True,
        f"Registros: {summary_199.get('total_registros', 'N/A')}"
    ))
    
    # Mostrar contenido clave
    print(f"\n📄 Contenido del JSON:")
    for key, value in summary_199.items():
        if key != "precio_m2":  # Evitar dict anidado
            print(f"   {key}: {value}")
        else:
            print(f"   precio_m2:")
            for k2, v2 in value.items():
                print(f"      {k2}: {v2}")
else:
    resultados_traza.append(ValidationResult(
        "JSON resumen generado",
        "extraction_summary_199.json",
        "No encontrado",
        False,
        f"No existe {SUMMARY_JSON}"
    ))

# 3.2 Log detallado
if LOG_FILE.exists():
    log_size = LOG_FILE.stat().st_size
    resultados_traza.append(ValidationResult(
        "Log detallado",
        "ine_extraction.log",
        f"{log_size} bytes",
        True,
        "✓ Trazabilidad completa"
    ))
else:
    resultados_traza.append(ValidationResult(
        "Log detallado",
        "ine_extraction.log",
        "No encontrado",
        False,
        "⚠️ Sin log detallado"
    ))

for r in resultados_traza:
    print(r)

# ==============================================================================
# 4. RESUMEN FINAL: DoD (Definition of Done)
# ==============================================================================

print("\n" + "=" * 80)
print("📊 RESUMEN: CRITERIOS DE ACEPTACIÓN (DoD) - ISSUE #199")
print("=" * 80)

# Consolidar todos los resultados
todos_resultados = resultados_infra + resultados_199 + resultados_traza

# Contar cumplimientos
total_criterios = len(todos_resultados)
cumplidos = sum(1 for r in todos_resultados if r.cumple)
pct_cumplimiento = (cumplidos / total_criterios) * 100

print(f"\n✅ Cumplidos: {cumplidos}/{total_criterios} ({pct_cumplimiento:.1f}%)")
print(f"❌ Pendientes: {total_criterios - cumplidos}")

# Tabla resumen
print("\n" + "-" * 80)
print(f"{'CRITERIO':<40} {'TARGET':<20} {'RESULTADO':<15} {'✓/✗'}")
print("-" * 80)

for r in todos_resultados:
    icono = "✅" if r.cumple else "❌"
    resultado_str = str(r.resultado)[:15]
    print(f"{r.criterio:<40} {str(r.target):<20} {resultado_str:<15} {icono}")

print("-" * 80)

# ==============================================================================
# 5. DECISIÓN GO/NO-GO
# ==============================================================================

print("\n" + "=" * 80)
print("🎯 DECISIÓN: GO/NO-GO PARA ISSUE #200 (Catastro)")
print("=" * 80)

# Criterios mínimos para continuar
criterios_minimos = {
    "volumen_100": df_spike is not None and len(df_spike) >= 100,
    "años_correctos": años_ok,
    "csv_generado": GRACIA_CSV_NOTEBOOK.exists(),
    "barrios_presentes": num_barrios >= 4
}

go_continuar = all(criterios_minimos.values())

if go_continuar:
    print("✅ GO - Todos los criterios mínimos cumplidos")
    print("   ➡️  Proceder con ISSUE #200: Extract Catastro")
    print(f"   📊 Dataset listo: {len(df_spike)} registros, {num_barrios} barrios, {len(años_unicos)} años")
else:
    print("❌ NO-GO - Criterios mínimos no cumplidos")
    print("   ⚠️  Revisar los siguientes puntos:")
    for criterio, cumple in criterios_minimos.items():
        if not cumple:
            print(f"      • {criterio}: {cumple}")
    print("\n   🔧 Acciones sugeridas:")
    print("      1. Re-ejecutar extract_precios_gracia.py con debug")
    print("      2. Verificar PORTALDADES_CLIENT_ID en .env")
    print("      3. Validar dim_barrios tiene barrios de Gràcia")

print("=" * 80)

# ==============================================================================
# 6. EXPORT PARA DOCUMENTACIÓN
# ==============================================================================

# Crear reporte markdown
reporte_md = f"""# Validación Spike Gràcia - Issue #199

**Fecha**: {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}  
**Cumplimiento global**: {pct_cumplimiento:.1f}% ({cumplidos}/{total_criterios})

## Resultados por Categoría

### 1. Infraestructura Reutilizada
"""

for r in resultados_infra:
    icono = "✅" if r.cumple else "❌"
    reporte_md += f"- {icono} **{r.criterio}**: {r.resultado} ({r.detalles})\n"

reporte_md += "\n### 2. Extracción Portal Dades (Issue #199)\n"
for r in resultados_199:
    icono = "✅" if r.cumple else "❌"
    reporte_md += f"- {icono} **{r.criterio}**: {r.resultado} ({r.detalles})\n"

reporte_md += "\n### 3. Trazabilidad\n"
for r in resultados_traza:
    icono = "✅" if r.cumple else "❌"
    reporte_md += f"- {icono} **{r.criterio}**: {r.resultado} ({r.detalles})\n"

reporte_md += f"""
## Decisión Final

**Status**: {'✅ GO' if go_continuar else '❌ NO-GO'}

### Próximos Pasos
{'- Proceder con Issue #200: Extract Catastro' if go_continuar else '- Revisar criterios fallidos antes de continuar'}
- Documentar findings en PRD
- Preparar seed CSV de referencias catastrales
"""

# Guardar reporte
reporte_path = SPIKE_DIR / "data/logs/validation_report_199.md"
reporte_path.parent.mkdir(parents=True, exist_ok=True)

with open(reporte_path, "w", encoding="utf-8") as f:
    f.write(reporte_md)

print(f"\n📝 Reporte guardado: {reporte_path}")


Existe CSV notebook: True
🔍 VALIDACIÓN 1: INFRAESTRUCTURA REUTILIZADA
✅ PortalDadesExtractor usado: 141 archivos en /Users/adrianiraeguialvear/Projects/barcelona-housing-demographics-analyzer/data/raw/portaldades (target: CSVs descargados)
✅ prepare_portaldades_precios ejecutado: CSV generado: ine_precios_gracia_notebook.csv (target: Datos procesados)
✅ dim_barrios cargado: 5 barrios encontrados (desde notebook) (target: 5 barrios Gràcia)

🔍 VALIDACIÓN 2: EXTRACCIÓN PORTAL DADES (ISSUE #199)
✅ CSV cargado: 1268 registros
✅ JSON resumen cargado: 1268 registros
✅ Período temporal: [np.int64(2020), np.int64(2021), np.int64(2022), np.int64(2023), np.int64(2024), np.int64(2025)] (target: 2020-2025)
✅ Cobertura barrios: 5 barrios ([np.int64(28), np.int64(29), np.int64(30), np.int64(31), np.int64(32)]) (target: 5 barrios)
✅ Columnas requeridas: 7 columnas: ['barrio_id', 'trimestre', 'anio', 'source', 'dataset_id']... (target: {'barrio_id', 'anio', 'precio_m2'})
✅ Calidad datos (nulos): 0.0% m